In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

RANDOM_STATE = 1234
%matplotlib inline

In [2]:
#Reading the data
columns = ["buying", "maint", "doors", "persons", "lug-boot", "safety", "car acceptability"]
dataset = pd.read_csv('D:/Work/car/Data/car.data', names=columns)
dataset.head()

,buying,maint,doors,persons,lug-boot,safety,car acceptability
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [3]:
y = pd.get_dummies(data=dataset['car acceptability'], 
                   prefix=['car acceptability'], 
                   columns=['car acceptability']).astype(int)
output_category = y.to_numpy()

In [4]:
#creating dummy variables for categorical data
one_hot_cols = ['doors','persons','buying', 'maint', 'lug-boot', 'safety']
dataset_1= pd.get_dummies(data=dataset, 
                          columns=one_hot_cols,
                          prefix=one_hot_cols)
dataset_1.head()

,car acceptability,doors_2,doors_3,doors_4,doors_5more,persons_2,persons_4,persons_more,buying_high,buying_low,...,maint_high,maint_low,maint_med,maint_vhigh,lug-boot_big,lug-boot_med,lug-boot_small,safety_high,safety_low,safety_med
0,unacc,True,False,False,False,True,False,False,False,False,...,False,False,False,True,False,False,True,False,True,False
1,unacc,True,False,False,False,True,False,False,False,False,...,False,False,False,True,False,False,True,False,False,True
2,unacc,True,False,False,False,True,False,False,False,False,...,False,False,False,True,False,False,True,True,False,False
3,unacc,True,False,False,False,True,False,False,False,False,...,False,False,False,True,False,True,False,False,True,False
4,unacc,True,False,False,False,True,False,False,False,False,...,False,False,False,True,False,True,False,False,False,True


In [5]:
# creating input features
X = dataset_1.iloc[:,[1,2,3,5,6,8,9,10,12,13,14,16,17]].astype(int)
X.head()

,doors_2,doors_3,doors_4,persons_2,persons_4,buying_high,buying_low,buying_med,maint_high,maint_low,maint_med,lug-boot_big,lug-boot_med
0,1,0,0,1,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,0,0,0
2,1,0,0,1,0,0,0,0,0,0,0,0,0
3,1,0,0,1,0,0,0,0,0,0,0,0,1
4,1,0,0,1,0,0,0,0,0,0,0,0,1


In [7]:
# creating training and test data
X_train, X_test, y_train, y_test = train_test_split(X, output_category, test_size=0.3)

In [18]:
output_category.shape

(1728, 4)

In [8]:
def build_classifier(optimizer='adam', kernel='random_uniform', units=4):
  classifier = Sequential()
  #First Hidden Layer
  classifier.add(Dense(units=units, activation='relu', kernel_initializer=kernel, input_dim=13))
  classifier.add(Dropout(rate=0.1))
  #Second  Hidden Layer
  classifier.add(Dense(units=units, activation='relu', kernel_initializer=kernel))
  #Output Layer
  classifier.add(Dense(4, activation='softmax', kernel_initializer=kernel))
  #Compiling the neural network
  classifier.compile(optimizer=optimizer,loss='categorical_crossentropy', metrics =['accuracy'])
  
  return classifier

In [9]:
classifier = KerasClassifier(model=build_classifier,
                             optimizer=None,
                             kernel=None,
                             units=None,
                             epochs=None,
                             batch_size=None)

In [10]:
param_grid = {
  "batch_size": [5, 10],
  "epochs": [10, 20, 50],
  "optimizer": ['adam', 'rmsprop', 'SGD'],
  "units": [4, 8, 13],
  "kernel": ['random_uniform']
}

gridSearch = GridSearchCV(estimator=classifier,
                          param_grid=param_grid,
                          cv=3,
                          return_train_score=True,
                          verbose=1)

In [11]:
X.shape

(1728, 13)

In [13]:
output_category.shape

(1728, 4)

In [16]:
X_train

,doors_2,doors_3,doors_4,persons_2,persons_4,buying_high,buying_low,buying_med,maint_high,maint_low,maint_med,lug-boot_big,lug-boot_med
624,0,0,0,1,0,1,0,0,1,0,0,0,1
194,0,0,0,1,0,0,0,0,1,0,0,0,1
773,1,0,0,0,1,1,0,0,0,1,0,1,0
1136,0,0,1,1,0,0,0,1,0,0,1,0,0
472,0,1,0,0,1,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1603,0,0,0,0,1,0,1,0,0,0,1,0,0
161,0,1,0,0,0,0,0,0,1,0,0,1,0
339,1,0,0,0,1,0,0,0,0,1,0,1,0
1360,0,0,1,0,1,0,1,0,0,0,0,0,0


In [12]:
gridSearch.fit(X, output_category)

Fitting 3 folds for each of 54 candidates, totalling 162 fits


Epoch 1/10
231/231 [==============================] - 1s 2ms/step - loss: 1.2671 - accuracy: 0.6163
Epoch 2/10
231/231 [==============================] - 0s 2ms/step - loss: 0.9855 - accuracy: 0.6189
Epoch 3/10
231/231 [==============================] - 0s 2ms/step - loss: 0.9210 - accuracy: 0.6189
Epoch 4/10
231/231 [==============================] - 0s 2ms/step - loss: 0.8754 - accuracy: 0.6189
Epoch 5/10
231/231 [==============================] - 0s 2ms/step - loss: 0.8431 - accuracy: 0.6198
Epoch 6/10
231/231 [==============================] - 0s 2ms/step - loss: 0.8168 - accuracy: 0.6189
Epoch 7/10
231/231 [==============================] - 0s 2ms/step - loss: 0.8047 - accuracy: 0.6094
Epoch 8/10
231/231 [==============================] - 0s 2ms/step - loss: 0.7965 - accuracy: 0.5981
Epoch 9/10
231/231 [==============================] - 0s 2ms/step - loss: 0.7661 - accuracy: 0.6155
Epoch 10/10
231/231 [==============================] - 0s 1ms/step
Epoch 1/10


KeyboardInterrupt: 

In [ ]:
gridSearch.best_params_

{'batch_size': 5,
 'epochs': 10,
 'kernel': 'random_uniform',
 'optimizer': 'adam',
 'units': 4}

In [ ]:
gridSearch.best_score_

0.7002314814814815

In [ ]:
df = pd.DataFrame(gridSearch.cv_results_)
df = df.sort_values("rank_test_score")
df.to_csv("result.csv")

### RandomizedSearchCV

In [ ]:
parameters = {
  "batch_size": [5, 10],
  "epochs": [10, 20, 50],
  "optimizer": ['adam', 'rmsprop', 'SGD'],
  "units": [4, 8, 13],
  "kernel": ['random_uniform']
}

random_search= RandomizedSearchCV(estimator=classifier, 
                                  param_distributions=parameters,
                                  n_iter=20,
                                  cv=3,
                                  verbose=0)


In [ ]:
random_search.fit(X, output_category)

Epoch 1/10


c:\Users\nishi\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


231/231 [==============================] - 1s 2ms/step - loss: 1.0839 - accuracy: 0.6111
Epoch 2/10
231/231 [==============================] - 0s 2ms/step - loss: 0.8667 - accuracy: 0.6189
Epoch 3/10
231/231 [==============================] - 0s 2ms/step - loss: 0.7921 - accuracy: 0.6189
Epoch 4/10
231/231 [==============================] - 0s 2ms/step - loss: 0.7500 - accuracy: 0.6068
Epoch 5/10
231/231 [==============================] - 0s 2ms/step - loss: 0.7239 - accuracy: 0.5946
Epoch 6/10
231/231 [==============================] - 0s 2ms/step - loss: 0.7047 - accuracy: 0.5955
Epoch 7/10
231/231 [==============================] - 0s 2ms/step - loss: 0.6907 - accuracy: 0.5998
Epoch 8/10
231/231 [==============================] - 0s 2ms/step - loss: 0.6830 - accuracy: 0.5972
Epoch 9/10
231/231 [==============================] - 0s 2ms/step - loss: 0.6777 - accuracy: 0.6042
Epoch 10/10
116/116 [==============================] - 0s 1ms/step
Epoch 1/10


c:\Users\nishi\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


231/231 [==============================] - 1s 2ms/step - loss: 1.0609 - accuracy: 0.7214
Epoch 2/10
231/231 [==============================] - 0s 2ms/step - loss: 0.7688 - accuracy: 0.7248
Epoch 3/10
231/231 [==============================] - 0s 1ms/step - loss: 0.6859 - accuracy: 0.7248
Epoch 4/10
231/231 [==============================] - 0s 1ms/step - loss: 0.6402 - accuracy: 0.7248
Epoch 5/10
231/231 [==============================] - 0s 2ms/step - loss: 0.6166 - accuracy: 0.7248
Epoch 6/10
231/231 [==============================] - 0s 2ms/step - loss: 0.6001 - accuracy: 0.7248
Epoch 7/10
231/231 [==============================] - 0s 1ms/step - loss: 0.5892 - accuracy: 0.7248
Epoch 8/10
231/231 [==============================] - 0s 1ms/step - loss: 0.5771 - accuracy: 0.7153
Epoch 9/10
231/231 [==============================] - 0s 1ms/step - loss: 0.5681 - accuracy: 0.6918
Epoch 10/10
116/116 [==============================] - 0s 1ms/step
Epoch 1/10


...

c:\Users\nishi\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


116/116 [==============================] - 2s 2ms/step - loss: 1.3141 - accuracy: 0.7500
Epoch 2/20
116/116 [==============================] - 0s 2ms/step - loss: 0.8584 - accuracy: 0.7569
Epoch 3/20
116/116 [==============================] - 0s 4ms/step - loss: 0.6343 - accuracy: 0.7569
Epoch 4/20
116/116 [==============================] - 0s 2ms/step - loss: 0.6020 - accuracy: 0.7569
Epoch 5/20
116/116 [==============================] - 0s 2ms/step - loss: 0.5845 - accuracy: 0.7569
Epoch 6/20
116/116 [==============================] - 0s 2ms/step - loss: 0.5750 - accuracy: 0.7569
Epoch 7/20
116/116 [==============================] - 0s 2ms/step - loss: 0.5688 - accuracy: 0.7569
Epoch 8/20
116/116 [==============================] - 0s 2ms/step - loss: 0.5545 - accuracy: 0.7569
Epoch 9/20
116/116 [==============================] - 0s 2ms/step - loss: 0.5482 - accuracy: 0.7569
Epoch 10/20
116/116 [==============================] - 0s 2ms/step - loss: 0.5355 - accuracy: 0.7569
Epoch 11/2

c:\Users\nishi\AppData\Local\Programs\Python\Python311\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


173/173 [==============================] - 1s 2ms/step - loss: 1.1079 - accuracy: 0.6985
Epoch 2/10
173/173 [==============================] - 0s 2ms/step - loss: 0.8005 - accuracy: 0.7002
Epoch 3/10
173/173 [==============================] - 0s 2ms/step - loss: 0.7369 - accuracy: 0.7002
Epoch 4/10
173/173 [==============================] - 0s 2ms/step - loss: 0.6858 - accuracy: 0.6997
Epoch 5/10
173/173 [==============================] - 0s 2ms/step - loss: 0.6536 - accuracy: 0.6910
Epoch 6/10
173/173 [==============================] - 0s 3ms/step - loss: 0.6305 - accuracy: 0.6892
Epoch 7/10
173/173 [==============================] - 0s 2ms/step - loss: 0.6072 - accuracy: 0.6834
Epoch 8/10
173/173 [==============================] - 0s 2ms/step - loss: 0.5945 - accuracy: 0.6863
Epoch 9/10
173/173 [==============================] - 0s 2ms/step - loss: 0.5913 - accuracy: 0.6840
Epoch 10/10
173/173 [==============================] - 0s 2ms/step - loss: 0.5868 - accuracy: 0.6852


RandomizedSearchCV(cv=3,
                   estimator=KerasClassifier(build_fn=<function build_classifier at 0x0000028EEFBF9DA0>, epochs=None, kernel=None, optimizer=None, units=None),
                   n_iter=20,
                   param_distributions={'batch_size': [5, 10],
                                        'epochs': [10, 20, 50],
                                        'kernel': ['random_uniform'],
                                        'optimizer': ['adam', 'rmsprop', 'SGD'],
                                        'units': [4, 8, 13]})

In [ ]:
random_search.best_params_

{'units': 8,
 'optimizer': 'adam',
 'kernel': 'random_uniform',
 'epochs': 10,
 'batch_size': 10}

In [ ]:
random_search.best_score_

0.6973379629629629